In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle
import pymde
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding, SpectralEmbedding
from scipy import sparse

import mlrfit as mf
import lrrouting as ldr

import cvxpy as cp
import numba as nb

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
rank = 6

mtype = "small_world"
n = 5000
# G = nx.connected_watts_strogatz_graph(n, k=4, p=0.1)
# G.remove_edges_from(nx.selfloop_edges(G))
# G = nx.DiGraph(G)

beta = 0.7
gamma = 0.01
G = nx.scale_free_graph(n, alpha=1-beta-gamma, beta=beta, gamma=gamma)#alpha=0.41, beta=0.54)

n = G.number_of_nodes()
print(f"{n=}, {G.number_of_edges()=}")

# for u, v in G.edges():
#     G[u][v]['weight'] = np.random.rand() * 10

Adj, Dist, nodes_cc = ldr.nx_graph_to_matrices(G, nodes=True)
G = G.subgraph(nodes_cc)
n = G.number_of_nodes()
A = Dist

n=5000, G.number_of_edges()=17057
[421, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [4]:
# ldr.plot_nx_G(G, with_labels=False, node_size=1, f_layout=nx.spring_layout, width=0.05)

In [5]:
assert nx.is_strongly_connected(G)
np.histogram(Dist.flatten(), bins=5, density=True)

(array([0.00500166, 0.24720014, 0.21925514, 0.02687584, 0.00166722]),
 array([ 0.,  2.,  4.,  6.,  8., 10.]))

In [6]:
adjacency_list = ldr.adjacency_directed_list(Adj)
sources, targets = ldr.st_pairs(n, Dist, 1020)
M = min(1000, sources.size)
sources = sources[:M]
targets = targets[:M]

In [7]:
PSD = False
w_min = A[A>0].min()
rt_max_iters = min(int(5*A.max()/w_min), (10**4) // 2)
symm = np.allclose(A, A.T)
print(f"{symm=}")
filename = "%s_r%d_%d"%(mtype, rank, n)

symm=False


In [8]:
np.histogram(Adj[Adj>0], bins=5, density=True)

(array([1.47168644e-02, 1.61990802e-05, 0.00000000e+00, 8.09954011e-06,
        8.09954011e-06]),
 array([  1. ,  68.8, 136.6, 204.4, 272.2, 340. ]))

In [9]:
info = {} 

In [10]:
fraction_of_nodes = 0.5
pi_rows = np.random.permutation(n)[:int(n * fraction_of_nodes)]
pi_cols = np.random.permutation(n)[:int(n * fraction_of_nodes)+10]

pi_row_Dist = ldr.sparse_sampled_matrix(pi_rows, Dist, rows=True)
pi_col_Dist_T = ldr.sparse_sampled_matrix(pi_cols, Dist, rows=False)

rDist = np.zeros((n, n))
rDist[pi_rows] = Dist[pi_rows]
assert np.allclose(pi_row_Dist.toarray(), rDist)

cDist = np.zeros((n, n))
cDist[:, pi_cols] = Dist[:, pi_cols]
assert np.allclose(pi_col_Dist_T.T.toarray(), cDist)
print("PASSED")

rDist = Dist[pi_rows]
cDist = Dist[:, pi_cols].T

pi_rows_c = np.delete(np.arange(n), pi_rows)
pi_cols_c = np.delete(np.arange(n), pi_cols)

PASSED


# convex concave slow and fast implementations

In [11]:
A0 = np.random.randn(2*n, 2*n)
A = A0 @ A0.T + 1e-5 * np.eye(2*n)
Z = np.random.randn(2*n, 20)

In [12]:
D_minsqrt = np.power(np.diag(A), -0.5)
N = (D_minsqrt[:, np.newaxis] * A) * D_minsqrt[np.newaxis, :]

assert np.allclose(N, np.diag(D_minsqrt) @ A @ np.diag(D_minsqrt)) and np.allclose(np.ones(A.shape[0]), np.diag(N))

N_inv = np.linalg.pinv(N, hermitian=True)

x2 = np.diag(D_minsqrt) @ (N_inv @  np.diag(D_minsqrt) @ Z)
x3 = D_minsqrt[:, np.newaxis] * (N_inv @  (D_minsqrt[:, np.newaxis] * Z))
x_true = np.linalg.solve(A, Z)

assert np.allclose(x_true, x2) and np.allclose(x_true, x3)

print("PASSED")

PASSED


In [13]:
Z, loss = ldr.slow_cc(rank, pi_rows, pi_cols, rDist, cDist, n_init=2, max_iter=500, eps=1e-6, verbose=True, freq=200, debug=True)[:2]

t=0, loss=6.01385891913236
t=200, loss=0.3113263698054456
t=400, loss=0.3099886239298567
loss=0.3097257161909192
t=0, loss=6.2856654158980065
t=200, loss=0.3104622571963844
t=400, loss=0.310039481169276
loss=0.3099690259328786


In [14]:
loss, rank, n

(0.3097257161909192, 6, 421)

In [15]:
l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
_ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}

median_stretch=100.0%, mean_stretch=252.3%
%[ratio<2] = 74.20%, %[ratio<1.2] = 62.40%, %[ratio=1.] = 61.50%


In [16]:
Z_rand = np.random.randn(Z.shape[0], Z.shape[1])
l_dar = ldr.construct_xy_node_embedding_graph(Z_rand[:n], Z_rand[n:], adjacency_list)
_ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}

median_stretch=250.0%, mean_stretch=385.0%
%[ratio<2] = 45.40%, %[ratio<1.2] = 25.00%, %[ratio=1.] = 23.60%


In [17]:
for _ in range(10):
    Z0 = np.random.randn(2*n, rank//2)
    X, Y = Z0[:n], Z0[n:]

    d_X = np.einsum('ik,ik->i', X, X)
    d_Y = np.einsum('ik,ik->i', Y, Y)

    res = ldr.product_BZ(X, Y, d_X, d_Y, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist)
    B = ldr.B_matrix(Z0, n, rDist, cDist, pi_rows, pi_cols)
    res_true = B @ Z0
    assert np.allclose(res, res_true), print(mf.rel_diff(res, res_true))
print("PASSED")

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


PASSED


In [18]:
dist_r, dist_c = ldr.pi_asymm_eulidean_dist_matrix(Z0[:n], Z0[n:], pi_rows, pi_cols)
dist_ij = ldr.asymm_eulidean_dist_matrix(Z0[:n], Z0[n:])
assert np.allclose(dist_r, dist_ij[pi_rows]) and np.allclose(dist_c, dist_ij[:, pi_cols].T)
print("PASSED") 

PASSED


In [19]:
Z, loss = ldr.fast_cc(rank, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist, n_init=2,
                   max_iter=1000, eps=1e-6, verbose=True, freq=100)[:2]

/Users/parshakova.tanya/Documents/projects/lr_distributed_routing/lrrouting/cg.py:128: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  rho0 = np.dot(r, z)
/Users/parshakova.tanya/Documents/projects/lr_distributed_routing/lrrouting/cg.py:152: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  losses[k] = np.sqrt(np.dot(r, r)) / b_norm


t=0, loss=5.832241208139824
t=100, loss=0.3165699193644933
t=200, loss=0.3104030834741061
t=300, loss=0.3091344071432101
t=400, loss=0.3083151330040682
t=500, loss=0.30779239796702
t=600, loss=0.3073809441502419
t=700, loss=0.3069935619389072
t=800, loss=0.3064941107515684
t=900, loss=0.30590269867184505
loss=0.3050104522809646, np.diff(np.array(losses)).max()=-3.75904304966701e-06
t=0, loss=6.081017613301836
t=100, loss=0.4048839529271361
t=200, loss=0.30208162673957983
t=300, loss=0.28825019594039347
t=400, loss=0.2833680864822589
t=500, loss=0.2809421416788004
t=600, loss=0.2795016956068462
t=700, loss=0.2785947933292568
t=800, loss=0.27798168240348664
t=900, loss=0.2775409101304055
loss=0.2772221020580667, np.diff(np.array(losses)).max()=-2.7109200889108287e-06


In [20]:
l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
_ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}

median_stretch=100.0%, mean_stretch=249.9%
%[ratio<2] = 76.50%, %[ratio<1.2] = 64.10%, %[ratio=1.] = 63.30%
